CODE TRANSLATION R_TO_PYTHON SECTION 1-10

Index:
1) ORGANISE THE DATA (RUN IN HPC)
2) COMPARE EMISSION FACTORS ACROSS TIERS
3) ESTIMATE THE NUMBER OF FARM DAMS IN 2015
4) GET THE STATISTICS ON SURFACE AREA FOR THE LAST FINANCIAL YEAR AT EACH MONTH FOR EACH STATE AND TERRITORY
5) GET HISTORICAL TRENDS
6) CALCULATE STATISTICS AND RATES
7) PLOTS AND SAVE DATA
8) PLOT THE FULL DATASET
9) ALL STATISTICS
10) SOURCES OF UNCERTAINTY

SECTION 1

In [16]:
import geopandas as gpd
import rasterio
import pandas as pd
from pandas.api.types import CategoricalDtype
import numpy as np
from datetime import datetime
import os
from shapely import wkt

In [2]:
# a) Load maps
AusDams = gpd.read_file("Required data/AusDams.geojson")
AUSstates = gpd.read_file("Required data/States Map_cropped.shp")
AusClimate = rasterio.open("Required data/clim-zones.tif")

In [9]:
Data = pd.read_csv("working/Dam forecast preds v6.csv")

In [17]:
Data['geometry'] = Data['geometry'].apply(wkt.loads)
Data = gpd.GeoDataFrame(Data, geometry='geometry')

In [18]:
# Nick's data
# Assuming the file "full_preds_v5.csv" has been loaded into a DataFrame named "Data"

# Add the locations
Data['x'] = Data['geometry'].apply(lambda geom: geom.x)
Data['y'] = Data['geometry'].apply(lambda geom: geom.y)

# Get the climate for each farm dam
Data['ClimateCode'] = Data.apply(lambda row: AusClimate.read(1, window=Data.bounds, masked=True)[row.y, row.x], axis=1)

# Add the state where the point comes from
e = AusDams.total_bounds
r = rasterio.Raster(e, height=1000, width=1000)
AUSstates_f = gpd.overlay(AUSstates, gpd.GeoDataFrame({'geometry': r.features()}, geometry='geometry'))
Data = gpd.sjoin(Data, AUSstates_f, how='left', op='within')

# Clean up and reformat Nick's data
Data2 = Data[(Data['area'] > 100) & (Data['area'] < 80000)]
Data2 = Data2[Data2.filter(like='X').gt(0).all(axis=1)]

# Pivot data
Data3 = pd.melt(Data2, id_vars=['State.ID', 'x', 'y', 'area', 'area_2', 'NAME', 'geometry'],
                value_vars=Data2.filter(like='X').columns,
                var_name='Date', value_name='Area')
Data3['Date'] = pd.to_datetime(Data3['Date'].str.replace('X', '').str.replace('.00.00.00', ''), format='%Y.%m.%d')

# Create FinYear
Data3['FinYear'] = (Data3['Date'] + pd.DateOffset(months=6)).dt.year.astype(str)

# d) Tier 2 approach
Tier2EF_dict = pd.read_excel("Required data/EF and climate zone dictionary.xlsx", dtype=str)
Data4 = Data3.merge(Tier2EF_dict, on='ClimateCode', how='left')

# e) Tier 3 approach
Temp2 = pd.read_pickle("Required data/Temp.pkl")
Temp2 = pd.melt(Temp2, id_vars=['X', 'Y'], value_vars=Temp2.filter(like='X').columns,
                var_name='Date', value_name='Temp')
Temp2['Date'] = pd.to_datetime(Temp2['Date'].str.replace('X', '').str.replace('_tav', '01'), format='%Y%m%d')
Data5 = pd.merge(Data4, Temp2, on=['x', 'y', 'Date'], how='left')

# f) Run the analyses
Boltzmann_const_evK = 8.617333262145 * 1e-5
FarmDams_CH4kgyearha_at15C = 365.25 * 10000 * (10 ** TotMet_15C_lm.params[0]) / 1e+6
FarmDams_CH4kgyearha_at15C_LCI = 365.25 * 10000 * (10 ** TotMet_15C_lm.conf_int()[0, 0]) / 1e+6
FarmDams_CH4kgyearha_at15C_UCI = 365.25 * 10000 * (10 ** TotMet_15C_lm.conf_int()[1, 0]) / 1e+6
Emissions_LCI = 100 * (FarmDams_CH4kgyearha_at15C - FarmDams_CH4kgyearha_at15C_LCI) / FarmDams_CH4kgyearha_at15C
Emissions_UCI = 100 * (FarmDams_CH4kgyearha_at15C_UCI - FarmDams_CH4kgyearha_at15C) / FarmDams_CH4kgyearha_at15C
Emissions_CV = np.mean(np.concatenate([Emissions_LCI, Emissions_UCI]))

# Correct for temperature
Data5['Boltzmann_15C'] = (1 / (Boltzmann_const_evK * (273.15 + 15))) - \
                         (1 / (Boltzmann_const_evK * (273.15 + Data5['Temp'])))

# Load the temp sensitivity parameter
Boltzmann_lm = pd.read_pickle("Required data/Boltzmann.pkl")
Data5['EF_T3'] = np.exp(Boltzmann_lm.params[1] * Data5['Boltzmann_15C'] +
                        np.log(FarmDams_CH4kgyearha_at15C))
Data5['EF_T3_LCI'] = np.exp(Boltzmann_lm.conf_int()[0, 1] * Data5['Boltzmann_15C'] +
                            np.log(FarmDams_CH4kgyearha_at15C_LCI))
Data5['EF_T3_UCI'] = np.exp(Boltzmann_lm.conf_int()[1, 1] * Data5['Boltzmann_15C'] +
                            np.log(FarmDams_CH4kgyearha_at15C_UCI))

# Calculate EF2 and 3 for each dam (kg CH4 per year per dam)
Data5['TotE_T3_kg_year'] = Data5['Area'] * Data5['EF_T3'] / 10000
Data5['TotE_T3_kg_year_LCI'] = (Data5['Area'] - 0.45 * Data5['Area']) * Data5['EF_T3_LCI'] / 10000
Data5['TotE_T3_kg_year_UCI'] = (Data5['Area'] + 0.45 * Data5['Area']) * Data5['EF_T3_UCI'] / 10000
Data5['TotE_T2_kg_year'] = Data5['Area'] * Data5['EF_T2'] / 10000

# g) Calculate the relative percentage capacity from Nick's data
Data2_T3 = Data5.copy()
Data2_T3['RelSurfaceArea'] = Data2_T3['Area'] / Data2_T3['area']

# Save smaller version of the file
Data2_T3_small = Data2_T3[['x', 'y', 'Date', 'ClimateCode', 'NAME', 'Area', 'FinYear', 'EF_T2',
                           'TotE_T2_kg_year', 'EF_T3', 'TotE_T3_kg_year', 'TotE_T3_kg_year_LCI',
                           'TotE_T3_kg_year_UCI']]
Data2_T3_small.to_pickle("Results/Data/Data2_T3_small_HPC.pkl")
Data2_T3.to_pickle("Results/Data/Data2_T3_HPC.pkl")

# Create a summary for the average water capacity in farm dams for each year
RelCapacity = Data2_T3.groupby(['Date', 'NAME', 'FinYear'])['RelSurfaceArea'].mean().reset_index()
RelCapacity.to_pickle("Results/Data/RelCapacity.pkl")


Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/home/nick/mambaforge/envs/gis/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 3508, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_2184271/831037287.py", line 9, in <module>
    Data['ClimateCode'] = Data.apply(lambda row: AusClimate.read(1, window=Data.bounds, masked=True)[row.y, row.x], axis=1)
                          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/nick/mambaforge/envs/gis/lib/python3.11/site-packages/geopandas/geodataframe.py", line 1581, in apply
    result = super().apply(
             ^^^^^^^^^^^^^^
  File "/home/nick/mambaforge/envs/gis/lib/python3.11/site-packages/pandas/core/frame.py", line 9423, in apply
    return op.apply().__finalize__(self, method="apply")
           ^^^^^^^^^^
  File "/home/nick/mambaforge/envs/gis/lib/python3.11/site-packages/pandas/core/apply.py", line 678, in apply

SECTION 2

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from scipy.stats import nbinom
from sklearn.linear_model import LinearRegression
from statsmodels.sandbox.regression.predstd import wls_prediction_std
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from matplotlib.patches import Patch

# 2) COMPARE EMISSION FACTORS ACROSS TIERS
if False:
    # Create the dataset
    TierComp = pd.DataFrame({'Temp': range(0, 51)})

    # Get temp-sensitivity
    Boltzmann_const_evK = 8.617333262145e-5

    # Get Boltzmann factor across temps
    TierComp['Boltzmann_15C'] = (1 / (Boltzmann_const_evK * (273.15 + 15))) - \
                                (1 / (Boltzmann_const_evK * (273.15 + TierComp['Temp'])))

    # Load the average farm dam emissions at 15C in mg.m2.day
    TotMet_15C = pd.read_csv("Required data/TotMet_15C.csv")
    FarmDams_CH4kgyearha_at15C = 365.25 * 10000 * (10 ** TotMet_15C['TotMet']) / 1e+6
    FarmDams_CH4kgyearha_at15C_LCI = 365.25 * 10000 * (10 ** TotMet_15C['TotMet_lower']) / 1e+6
    FarmDams_CH4kgyearha_at15C_UCI = 365.25 * 10000 * (10 ** TotMet_15C['TotMet_upper']) / 1e+6

    # Get emissions T3 (kt.year.ha)
    TierComp['EF_T3'] = np.exp(Boltzmann_lq[1] * TierComp['Boltzmann_15C'] +
                               np.log(FarmDams_CH4kgyearha_at15C)) * 1000 / 1000000
    TierComp['EF_T3_LCI'] = np.exp(Boltzmann_lq[1] * TierComp['Boltzmann_15C'] +
                                   np.log(FarmDams_CH4kgyearha_at15C_LCI)) * 1000 / 1000000
    TierComp['EF_T3_UCI'] = np.exp(Boltzmann_lq[1] * TierComp['Boltzmann_15C'] +
                                   np.log(FarmDams_CH4kgyearha_at15C_UCI)) * 1000 / 1000000

    # Get emissions T1 (kt.year.ha)
    TierComp['EF_T1'] = 183 * 1000 / 1000000
    TierComp['EF_T1_LCI'] = 118 * 1000 / 1000000
    TierComp['EF_T1_UCI'] = 228 * 1000 / 1000000

    # Get emissions T2, climate-specific EF following Tertius directions (Kg CH4 ha-1 year-1)
    Tier2EF_dict = pd.read_excel("Required data/EF and climate zone dictionary.xlsx", sheet_name="Restructure2")
    Tier2EF_dict['EF_T2.kt'] = Tier2EF_dict['EF_T2'] * 1000 / 1000000
    Tier2EF_dict['Climate'] = Tier2EF_dict['Climate'].astype('category')

    TierComp2 = TierComp.merge(Tier2EF_dict, on='Temp', how='outer')

    # Extract the CI from the relationship
    T2_raw = pd.read_excel("Required data/Farmdam EF values_COPY.xlsx", sheet_name="Working data set_R")
    T2_raw['EF_kg.CH4.ha.y'] = np.log10(T2_raw['EF_kg.CH4.ha.y'])
    T2_raw['Temp'] = T2_raw['Temp'] + 273.15

    def func(x, a, b):
        return a * x ** b

    popt, _ = curve_fit(func, T2_raw['Temp'], T2_raw['EF_kg.CH4.ha.y'])
    T2_raw['pred'] = func(T2_raw['Temp'], *popt)

    prdc = np.random.normal(loc=T2_raw['pred'], scale=0.1, size=(1000, len(T2_raw)))

    # Calculate prediction intervals
    prdc_CI = pd.DataFrame({
        'Temp': T2_raw['Temp'],
        'LCI.kt': 10 ** (np.percentile(prdc, 5, axis=1)) * 1000 / 1000000,
        'UCI.kt': 10 ** (np.percentile(prdc, 95, axis=1)) * 1000 / 1000000,
        'LCI': 10 ** (np.percentile(prdc, 5, axis=1)),
        'UCI': 10 ** (np.percentile(prdc, 95, axis=1))
    })

    prdc_CI['Pred.kt'] = 10 ** (np.round(prdc_CI['Temp'], 2)) * 1000 / 1000000
    prdc_CI['Pred'] = 10 ** (np.round(prdc_CI['Temp'], 2))

    # Check the fit
    plt.figure()
    plt.plot(T2_raw['Temp'], prdc.T, color='gray', alpha=0.5)
    plt.plot(T2_raw['Temp'], T2_raw['EF_kg.CH4.ha.y'], 'o')
    plt.plot(T2_raw['Temp'], T2_raw['pred'])
    plt.fill_between(prdc_CI['Temp'], prdc_CI['LCI'], prdc_CI['UCI'], color='blue', alpha=0.1)
    plt.show()

    # Log-10 scale
    plt.figure()
    plt.plot(T2_raw['Temp'], prdc.T, color='gray', alpha=0.5)
    plt.plot(T2_raw['Temp'], T2_raw['EF_kg.CH4.ha.y'], 'o')
    plt.plot(T2_raw['Temp'], np.log10(prdc_CI['Pred']))
    plt.fill_between(prdc_CI['Temp'], np.log10(prdc_CI['LCI']), np.log10(prdc_CI['UCI']), color='blue', alpha=0.1)
    plt.show()

    # Aritmnetic scale
    plt.figure()
    plt.plot(T2_raw['Temp'], 10 ** prdc.T, color='gray', alpha=0.5)
    plt.plot(T2_raw['Temp'], T2_raw['EF_kg.CH4.ha.y'], 'o')
    plt.plot(T2_raw['Temp'], prdc_CI['Pred'])
    plt.fill_between(prdc_CI['Temp'], prdc_CI['LCI'], prdc_CI['UCI'], color='blue', alpha=0.1)
    plt.show()

    # Check for patterns
    print(prdc_CI['LCI'] / prdc_CI['Pred'])
    print(prdc_CI['UCI'] / prdc_CI['Pred'])
    print(prdc_CI['Pred'] / prdc_CI['UCI'])

    # Averages
    print(np.mean(prdc_CI['LCI'] / prdc_CI['Pred']))
    print(np.mean(prdc_CI['UCI'] / prdc_CI['Pred']))

    # Solution:
    # For the LCI, I will multiply the predicted log-10 value by 0.8432157
    # For the UCI, I will multiply the predicted log-10 value by 1.100405

    TierComp2['EF_T2.kt_LCI'] = 10 ** (np.log10(TierComp2['EF_T2.kt']) * 0.8604673)
    TierComp2['EF_T2.kt_UCI'] = 10 ** (np.log10(TierComp2['EF_T2.kt']) * 1.1085)

    # Plot
    plt.figure()
    plt.plot(TierComp2.loc[TierComp2['Temp'] < 41, 'Temp'], TierComp2.loc[TierComp2['Temp'] < 41, 'EF_T3_LCI'],
             alpha=0.1, label='Tier 3 LCI')
    plt.plot(TierComp2.loc[TierComp2['Temp'] < 41, 'Temp'], TierComp2.loc[TierComp2['Temp'] < 41, 'EF_T3'],
             label='Tier 3', linewidth=1.5)
    plt.plot(TierComp2.loc[TierComp2['Temp'] < 41, 'Temp'], TierComp2.loc[TierComp2['Temp'] < 41, 'EF_T1_LCI'],
             alpha=0.1, label='Tier 1 LCI')
    plt.plot(TierComp2.loc[TierComp2['Temp'] < 41, 'Temp'], TierComp2.loc[TierComp2['Temp'] < 41, 'EF_T1'],
             label='Tier 1', linestyle='dashed', linewidth=1.5)

    unique_climates = TierComp2['Climate'].unique()
    color_map = plt.cm.get_cmap('viridis', len(unique_climates))

    for i, climate in enumerate(unique_climates):
        temp_range = TierComp2.loc[TierComp2['Temp'] < 41, 'Temp']
        ef_lci = TierComp2.loc[TierComp2['Temp'] < 41].loc[TierComp2['Climate'] == climate, 'EF_T2.kt_LCI']
        ef_uci = TierComp2.loc[TierComp2['Temp'] < 41].loc[TierComp2['Climate'] == climate, 'EF_T2.kt_UCI']
        plt.fill_between(temp_range, ef_lci, ef_uci, color=color_map(i), alpha=0.1)
        plt.plot(temp_range, TierComp2.loc[TierComp2['Temp'] < 41].loc[TierComp2['Climate'] == climate, 'EF_T2.kt'],
                 color=color_map(i), linewidth=1.5, label=f'Tier 2 {climate}')

    plt.ylim(0, 1)
    plt.xlim(0, 41)
    plt.xlabel('Mean annual temperature (°C)')
    plt.ylabel('Methane emission factors (EF; t CH₄ ha⁻¹ year⁻¹)')
    plt.legend()
    plt.title('Comparison of Emission Factors Across Tiers')
    plt.annotate('Temp-dependent EF', xy=(36, 0.84), xytext=(36, 0.84), size=8)
    plt.annotate('Temp-independent EF', xy=(36, 0.15), xytext=(36, 0.15), size=8)
    plt.show()

    # Save the plot as PDF
    plt.savefig('Results/Plot/Emission_factors.pdf', format='pdf', dpi=300)

    # Save the plot as JPEG
    plt.savefig('Results/Plot/Emission_factors.jpeg', format='jpeg', dpi=300)

SECTION 3

In [ ]:
import pandas as pd
import numpy as np
import dask.dataframe as dd
from dask.distributed import Client

# Create a Dask client for parallel processing
client = Client(n_workers=10)

# Load Data2.T3.small.HPC.RData
data2_t3_small = dd.read_parquet("Results/Data/Data2.T3.small.HPC.parquet")
data2_t3_small = data2_t3_small.compute()

# Get the final statistics from Malerba et al (2021)
counts_df = pd.read_csv("Required data/FinalOutcome.s.csv")
counts_df = counts_df[['State.f', 'Count.50', 'Count.2.5', 'Count.97.5']].rename(columns={'State.f': 'State'})

# Calculate the adjusted coefficients
counts_df['AdjCoef'] = counts_df['Count.50'] / data2_t3_small.groupby('NAME')['index'].nunique()
counts_df['AdjCoef.LCI'] = counts_df['Count.2.5'] / data2_t3_small.groupby('NAME')['index'].nunique()
counts_df['AdjCoef.UCI'] = counts_df['Count.97.5'] / data2_t3_small.groupby('NAME')['index'].nunique()

# Save the results
counts_df.to_csv("Results/Data/AllCounts.csv", index=False)
counts_df.to_pickle("Results/Data/AllCounts.pkl")

# AllCounts: Number of farm dams in 2015 from Malerba et al 2021 Remote Sensing
import dask.dataframe as dd

AllCounts = dd.read_csv("Results/Data/AllCounts.csv").compute()

# Close the Dask client
client.close()


SECTION 4

In [ ]:
import pandas as pd
import numpy as np
from dplython import (DplyFrame, X, select, group_by, summarize, arrange, left_join, merge, mutate, filter)

# Load data (change file paths accordingly)
Data2_T3_Small_HPC = pd.read_pickle("Results/Data/Data2.T3.Small.HPC.pkl")
AllCounts = pd.read_pickle("Results/Data/AllCounts.pkl")

# Define a function to calculate adjusted values
def calculate_adj_values(df):
    df['SumArea.kha.Adj'] = df['SumArea.kha'] * df['AdjCoef']
    df['SumArea.kha.Adj.LCI'] = df['SumArea.kha_LCI'] * df['AdjCoef.LCI']
    df['SumArea.kha.Adj.UCI'] = df['SumArea.kha_UCI'] * df['AdjCoef.UCI']
    df['TotE_T3.kt.year.Adj'] = df['TotE_T3.kt.year'] * df['AdjCoef']
    df['TotE_T3.kt.year.Adj.LCI'] = df['TotE_T3.kt.year_LCI'] * df['AdjCoef.LCI']
    df['TotE_T3.kt.year.Adj.UCI'] = df['TotE_T3.kt.year_UCI'] * df['AdjCoef.UCI']
    df['TotE_T2.kt.year.Adj'] = df['TotE_T2.kt.year'] * df['AdjCoef']
    return df

# Group by State, FinYear and calculate summary statistics
Data2_T3_kha = (
    Data2_T3_Small_HPC
    # Filter data here if needed
    .groupby(['NAME', 'FinYear'])
    .agg(
        SumArea_kha=pd.NamedAgg(column='Area', aggfunc='sum') * 1e-7,
        SumArea_kha_LCI=pd.NamedAgg(column='Area', aggfunc=lambda x: sum(x) * 1e-7 - 0.45 * sum(x) * 1e-7),
        SumArea_kha_UCI=pd.NamedAgg(column='Area', aggfunc=lambda x: sum(x) * 1e-7 + 0.45 * sum(x) * 1e-7),
        SumTotE_T3_kt_year=pd.NamedAgg(column='TotE_T3.kg.year', aggfunc='sum') / 1000000,
        SumTotE_T3_kt_LCI_year=pd.NamedAgg(column='TotE_T3.kg.year_LCI', aggfunc='sum') / 1000000,
        SumTotE_T3_kt_UCI_year=pd.NamedAgg(column='TotE_T3.kg.year_UCI', aggfunc='sum') / 1000000,
        SumTotE_T2_kt_year=pd.NamedAgg(column='TotE_T2.kg.year', aggfunc='sum') / 1000000
    )
    .reset_index()
    .rename(columns={'NAME': 'State'})
    .pipe(calculate_adj_values)
    .merge(AllCounts, on='State')
)

# Extract years with full data
TableYear = (Data2_T3_kha['FinYear'].value_counts() == 96).reset_index()
FullYear = TableYear.loc[TableYear['FinYear'], 'index']

# Keep only years with all observations
Data2_T3_kha = Data2_T3_kha[Data2_T3_kha['FinYear'].isin(FullYear)]

# Save data (change file path accordingly)
Data2_T3_kha.to_pickle("Results/Data/Data2.T3.kha.pkl")


In [ ]:
# Load necessary data (change file paths accordingly)
Data2_T3_kha = pd.read_pickle("Results/Data/Data2.T3.kha.pkl")
Data3_T3_kha = Data2_T3_kha.groupby(['State', 'FinYear']).agg(
    SumArea_kha=pd.NamedAgg(column='SumArea.kha', aggfunc='mean'),
    SumArea_kha_LCI=pd.NamedAgg(column='SumArea.kha_LCI', aggfunc='mean'),
    SumArea_kha_UCI=pd.NamedAgg(column='SumArea.kha_UCI', aggfunc='mean'),
    TotE_T3_kt_year=pd.NamedAgg(column='SumTotE_T3.kt.year', aggfunc='mean'),
    TotE_T3_kt_LCI_year=pd.NamedAgg(column='SumTotE_T3.kt.year.LCI', aggfunc='mean'),
    TotE_T3_kt_UCI_year=pd.NamedAgg(column='SumTotE_T3.kt.year.UCI', aggfunc='mean'),
    TotE_T2_kt_year=pd.NamedAgg(column='SumTotE_T2.kt.year', aggfunc='mean')
).reset_index()

Data3_T3_kha = Data3_T3_kha.merge(AllCounts, on='State')

# Calculate adjusted values
Data3_T3_kha = calculate_adj_values(Data3_T3_kha)

# Calculate NT relative density
NT_reldensity = AllCounts.loc[AllCounts['State'] == "Northern Territory", 'Count.50'].values[0] / AllCounts['Count.50'].sum()

# Calculate NT statistics
Data3_kha_NT = (
    Data3_T3_kha[Data3_T3_kha['State'] != "Northern Territory"]
    .groupby('FinYear')
    .agg(
        SumArea_kha_Adj=pd.NamedAgg(column='SumArea.kha.Adj', aggfunc='sum') * NT_reldensity,
        SumArea_kha_Adj_LCI=pd.NamedAgg(column='SumArea.kha.Adj.LCI', aggfunc='sum') * NT_reldensity,
        SumArea_kha_Adj_UCI=pd.NamedAgg(column='SumArea.kha.Adj.UCI', aggfunc='sum') * NT_reldensity,
        TotE_T2_kt_year_Adj=pd.NamedAgg(column='TotE_T2.kt.year.Adj', aggfunc='sum') * NT_reldensity,
        TotE_T3_kt_year_Adj=pd.NamedAgg(column='TotE_T3.kt.year.Adj', aggfunc='sum') * NT_reldensity,
        TotE_T3_kt_year_Adj_LCI=pd.NamedAgg(column='TotE_T3.kt.year.Adj.LCI', aggfunc='sum') * NT_reldensity,
        TotE_T3_kt_year_Adj_UCI=pd.NamedAgg(column='TotE_T3.kt.year.Adj.UCI', aggfunc='sum') * NT_reldensity
    )
    .reset_index()
    .assign(State="Northern Territory")
)

# Combine NT statistics with other states
Data3_kha_all = pd.concat([Data3_kha_NT, Data3_T3_kha[Data3_T3_kha['State'] != "Northern Territory"]])

# Calculate Australia-wide statistics
Data4_kha = (
    Data3_kha_all.groupby('FinYear')
    .agg(
        SumArea_kha_Adj=pd.NamedAgg(column='SumArea.kha.Adj', aggfunc='sum'),
        SumArea_kha_Adj_LCI=pd.NamedAgg(column='SumArea.kha.Adj.LCI', aggfunc='sum'),
        SumArea_kha_Adj_UCI=pd.NamedAgg(column='SumArea.kha.Adj.UCI', aggfunc='sum'),
        TotE_T2_kt_year_Adj=pd.NamedAgg(column='TotE_T2.kt.year.Adj', aggfunc='sum'),
        TotE_T3_kt_year_Adj=pd.NamedAgg(column='TotE_T3.kt.year.Adj', aggfunc='sum'),
        TotE_T3_kt_year_Adj_LCI=pd.NamedAgg(column='TotE_T3.kt.year.Adj.LCI', aggfunc='sum'),
        TotE_T3_kt_year_Adj_UCI=pd.NamedAgg(column='TotE_T3.kt.year.Adj.UCI', aggfunc='sum')
    )
    .reset_index()
    .assign(State="Australia")
)

# Combine Australia-wide statistics with other states
Data5_kha = pd.concat([Data4_kha, Data3_kha_all])

# Save the final datasets (change file paths accordingly)
Data3_T3_kha.to_pickle("Results/Data/Data3.T3.kha.pkl")
Data3_kha_NT.to_pickle("Results/Data/Data3.kha.NT.pkl")
Data3_kha_all.to_pickle("Results/Data/Data3.kha.all.pkl")
Data4_kha.to_pickle("Results/Data/Data4.kha.pkl")
Data5_kha.to_pickle("Results/Data/Data5.kha.pkl")

In [ ]:
# Load necessary data (change file paths accordingly)
Data3_T3_kha = pd.read_pickle("Results/Data/Data3.T3.kha.pkl")
#Data3_kha_NT = pd.read_pickle("Results/Data/Data3.kha.NT.pkl")
#Data3_kha_all = pd.read_pickle("Results/Data/Data3.kha.all.pkl")
Data4_kha = pd.read_pickle("Results/Data/Data4.kha.pkl")
Data5_kha = pd.read_pickle("Results/Data/Data5.kha.pkl")

SECTION 5

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns

# Load required data
# ====================
# MyECDF.single
my_ecdf_single = pd.read_csv("Required data/MyECDF.single.csv")
my_ecdf_single.rename(columns={"X2": "State"}, inplace=True)

# Create MyECDF.allyears
unique_states = my_ecdf_single["State"].unique()
unique_years = my_ecdf_single["YearBuilt"].unique()
my_ecdf_allyears = pd.DataFrame({'State': np.repeat(unique_states, len(unique_years)),
                                 'YearBuilt': np.tile(unique_years, len(unique_states))})

# Merge with MyECDF.single to fill missing values
my_ecdf_single = pd.merge(my_ecdf_allyears, my_ecdf_single, on=["State", "YearBuilt"], how="left")

# Replace missing values in Australian Capital Territory
act_indices = (my_ecdf_single["State"] == "Australian Capital Territory")
act_ecd = my_ecdf_single.loc[act_indices, "ecd"]
act_ecd = act_ecd.fillna(act_ecd.rolling(window=2).mean())  # Replace with a simple moving average
my_ecdf_single.loc[act_indices, "ecd"] = act_ecd

# Correct missing values in Australian Capital Territory for certain years
act_missing_years = [1995, 1996, 1997]
my_ecdf_single.loc[act_indices & my_ecdf_single["YearBuilt"].isin(act_missing_years), "ecd"] = \
    0.6393443 + np.diff([0.6393443, 0.6721311]) / 4 * np.array([1, 2, 3])

# Calculate ecd for Northern Territory based on overall mean for Australia
my_ecdf_single2 = my_ecdf_single.groupby("YearBuilt")["ecd"].mean().reset_index()
my_ecdf_single2["State"] = "Northern Territory"

# Merge with other States
my_ecdf_single3 = pd.concat([my_ecdf_single2, my_ecdf_single])

# Calculate the projected rate of increase after 2005 for each State
my_ecdf_single3_recent = my_ecdf_single3[my_ecdf_single3["YearBuilt"] > 2005]
relative_growth_lm = sm.OLS(my_ecdf_single3_recent["ecd"], sm.add_constant(my_ecdf_single3_recent[["YearBuilt", "State"]])).fit()
predict_data = pd.DataFrame({'State': unique_states,
                             'YearBuilt': np.arange(2005, 2021)})
predict_data["Pred"] = relative_growth_lm.predict(sm.add_constant(predict_data[["YearBuilt", "State"]]))

# Visualize historical trends
plt.figure(figsize=(10, 6))
sns.set(style="whitegrid")
g = sns.FacetGrid(data=my_ecdf_single3, col="State", col_wrap=3, height=4)
g.map(plt.step, "YearBuilt", "ecd", color='b', where="mid")
g.map(plt.plot, "YearBuilt", "Pred", color='r')
g.set_axis_labels("Year", "Historical increase in dam density")
g.set_titles(col_template="{col_name}")
g.add_legend()
plt.tight_layout()
plt.savefig("Results/Plot/Predict.Plot.pdf")

# Generate predictions for each State
final_predictions = pd.DataFrame({'State': unique_states,
                                 'YearBuilt': np.arange(2010, 2021)})
final_predictions["ecd.pred"] = relative_growth_lm.predict(sm.add_constant(final_predictions[["YearBuilt", "State"]]))

# Combine observations and predictions
hist_data = pd.merge(my_ecdf_single3, final_predictions, on=["State", "YearBuilt"], how="left")
hist_data["ecd.comb"] = np.where(hist_data["ecd"].isnull(), hist_data["ecd.pred"], hist_data["ecd"])
hist_data["ecd.adj"] = hist_data["ecd.comb"] / hist_data.groupby("State")["ecd.comb"].transform("max")

# Rename columns and convert FinYear to numeric
hist_data.rename(columns={"YearBuilt": "FinYear"}, inplace=True)
hist_data["FinYear"] = hist_data["FinYear"].astype(int)

# Load other data
data5_kha_hist = pd.read_csv("Results/Data/Data5.kha.hist.csv")
all_counts_aus = pd.read_csv("Results/Data/AllCounts.aus.csv")

# Merge data and perform necessary calculations
data6_kha_hist = pd.merge(data5_kha_hist, hist_data, on=["State", "FinYear"], how="left")
data6_kha_hist = data6_kha_hist.merge(all_counts_aus[["State", "Count.50"]], on="State", how="left")
data6_kha_hist["SumCount.Adj.ecd"] = data6_kha_hist["Count.50"] * data6_kha_hist["ecd.adj"]
data6_kha_hist["SumCount.Adj.LCI.ecd"] = data6_kha_hist["Count.50"] * 0.025 * data6_kha_hist["ecd.adj"]
data6_kha_hist["SumCount.Adj.UCI.ecd"] = data6_kha_hist["Count.50"] * 0.975 * data6_kha_hist["ecd.adj"]

# Whole Australia statistics by summing across the States and Territories
data6_kha_hist_aus = data6_kha_hist.groupby("FinYear").agg({"SumArea.kha.Adj.ecd": "sum",
                                                           "SumArea.kha.Adj.LCI.ecd": "sum",
                                                           "SumArea.kha.Adj.UCI.ecd": "sum",
                                                           "TotE_T3.kt.year.Adj.ecd": "sum",
                                                           "TotE_T3.kt.year.Adj.LCI.ecd": "sum",
                                                           "TotE_T3.kt.year.Adj.UCI.ecd": "sum",
                                                           "SumCount.Adj.ecd": "sum",
                                                           "SumCount.Adj.LCI.ecd": "sum",
                                                           "SumCount.Adj.UCI.ecd": "sum"}).reset_index()
data6_kha_hist_aus["State"] = "Australia"

# Save data
data6_kha_hist.to_csv("Results/Data/Data6.kha.hist.csv", index=False)
data6_kha_hist_aus.to_csv("Results/Data/Data6.kha.hist.Aus.csv", index=False)


# Load data at the end of the script
data6_kha_hist = pd.read_csv("Results/Data/Data6.kha.hist.csv")
data6_kha_hist_aus = pd.read_csv("Results/Data/Data6.kha.hist.Aus.csv")


SECTION 6

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns

# 6) PLOTTING PREDICTED INCREASES IN DAM DENSITY
# =============

# NOTE: SumCount.Adj.ecd are the predicted values based on observations and linear regression,
#       SumCount.ecd are only based on observations

# Calculate the projected rate of increase in total numbers after 2005 for each State
subset_data = Data6_kha_hist[Data6_kha_hist['FinYear'] > 2005]

X = sm.add_constant(subset_data[['FinYear', 'State']])
y = subset_data['SumCount.ecd']
abs_growth_model = sm.OLS(y, X).fit()

unique_states = Data6_kha_hist['State'].unique()
predict_data_abs = pd.DataFrame({'State': np.repeat(unique_states, 16), 'FinYear': np.tile(np.arange(2005, 2021), 8)})
predict_data_abs = sm.add_constant(predict_data_abs)
predict_data_abs['Pred'] = abs_growth_model.predict(predict_data_abs)

# Plot total farm dam numbers and projected increases
plt.figure(figsize=(9.5, 10))
sns.set_style("whitegrid")
for state in unique_states:
    state_data = Data6_kha_hist[Data6_kha_hist['State'] == state]
    plt.step(state_data['FinYear'], state_data['SumCount.ecd'] / 1000, where='mid', label=state, linewidth=2)
    predict_state = predict_data_abs[predict_data_abs['State'] == state]
    plt.plot(predict_state['FinYear'], predict_state['Pred'] / 1000, label=f'Predicted {state}', linewidth=2, linestyle='dashed')

plt.xlabel('Year')
plt.ylabel('Historical increase in dam numbers (thousands)')
plt.title('Predicted Increases in Farm Dam Density')
plt.legend()
plt.xlim(1988, 2020)
plt.grid(False)
plt.savefig('Results/Plot/PredictAbs.Plot.png')
plt.close()

# RESULTS: Calculate relative increase from 1988 to 2020
state_summary = []
for state in unique_states:
    counts_1988 = Data6_kha_hist[(Data6_kha_hist['State'] == state) & (Data6_kha_hist['FinYear'] == 1988)]['SumCount.Adj.ecd'].values[0]
    counts_2020 = Data6_kha_hist[(Data6_kha_hist['State'] == state) & (Data6_kha_hist['FinYear'] == 2020)]['SumCount.Adj.ecd'].values[0]
    rel_count_1988_to_2020 = (counts_2020 - counts_1988) / counts_1988
    sa_1988 = Data6_kha_hist[(Data6_kha_hist['State'] == state) & (Data6_kha_hist['FinYear'] == 1988)]['SumArea.kha.Adj.ecd'].values[0]
    sa_2020 = Data6_kha_hist[(Data6_kha_hist['State'] == state) & (Data6_kha_hist['FinYear'] == 2020)]['SumArea.kha.Adj.ecd'].values[0]
    rel_sa_1988_to_2020 = (sa_2020 - sa_1988) / sa_1988
    abs_count_1988_to_2020 = counts_2020 - counts_1988
    abs_count_LCI_1988_to_2020 = Data6_kha_hist[(Data6_kha_hist['State'] == state) & (Data6_kha_hist['FinYear'] == 2020)]['SumCount.Adj.LCI.ecd'].values[0] - \
                                  Data6_kha_hist[(Data6_kha_hist['State'] == state) & (Data6_kha_hist['FinYear'] == 1988)]['SumCount.Adj.LCI.ecd'].values[0]
    abs_count_UCI_1988_to_2020 = Data6_kha_hist[(Data6_kha_hist['State'] == state) & (Data6_kha_hist['FinYear'] == 2020)]['SumCount.Adj.UCI.ecd'].values[0] - \
                                  Data6_kha_hist[(Data6_kha_hist['State'] == state) & (Data6_kha_hist['FinYear'] == 1988)]['SumCount.Adj.UCI.ecd'].values[0]
    abs_sa_1988_to_2020 = sa_2020 - sa_1988
    abs_sa_LCI_1988_to_2020 = Data6_kha_hist[(Data6_kha_hist['State'] == state) & (Data6_kha_hist['FinYear'] == 2020)]['SumArea.kha.Adj.LCI.ecd'].values[0] - \
                              Data6_kha_hist[(Data6_kha_hist['State'] == state) & (Data6_kha_hist['FinYear'] == 1988)]['SumArea.kha.Adj.LCI.ecd'].values[0]
    abs_sa_UCI_1988_to_2020 = Data6_kha_hist[(Data6_kha_hist['State'] == state) & (Data6_kha_hist['FinYear'] == 2020)]['SumArea.kha.Adj.UCI.ecd'].values[0] - \
                              Data6_kha_hist[(Data6_kha_hist['State'] == state) & (Data6_kha_hist['FinYear'] == 1988)]['SumArea.kha.Adj.UCI.ecd'].values[0]
    state_summary.append([state, counts_1988, counts_2020, rel_count_1988_to_2020, sa_1988, sa_2020, rel_sa_1988_to_2020,
                          abs_count_1988_to_2020, abs_count_LCI_1988_to_2020, abs_count_UCI_1988_to_2020,
                          abs_sa_1988_to_2020, abs_sa_LCI_1988_to_2020, abs_sa_UCI_1988_to_2020])

state_summary_columns = ['State', 'Counts_1988', 'Counts_2020', 'Rel_Count_1988to2020', 'SA_1988', 'SA_2020', 'Rel_SA_1988to2020',
                         'Abs_Count_1988to2020', 'Abs_Count_LCI_1988to2020', 'Abs_Count_UCI_1988to2020',
                         'Abs_SA_1988to2020', 'Abs_SA_LCI_1988to2020', 'Abs_SA_UCI_1988to2020']

in_cr_df = pd.DataFrame(state_summary, columns=state_summary_columns)
in_cr_df.sort_values(by='Abs_Count_1988to2020', ascending=False, inplace=True)

in_cr_df.to_csv("Results/Data/Incr1988to2020.csv", index=False)
in_cr_df.to_pickle("Results/Data/Incr1988to2020.pkl")

# Incr1988to2020: Predicted increase from 1988 to 2020
in_cr_df = pd.read_pickle("Results/Data/Incr1988to2020.pkl")

#Please note that this translation assumes you have already loaded the Data6_kha_hist DataFrame earlier in your code.
#Adjust the code accordingly based on your specific setup.

SECTION 7

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from mizani.breaks import date_breaks
from mizani.formatters import date_format
from plotnine import ggplot, aes, geom_area, geom_line, labs, theme_bw, \
    facet_wrap, guides, scale_x_continuous, scale_y_continuous, sec_axis
from plotnine.arrange import plot_grid
from scipy.stats import pearsonr

# Load Rainfall Anomalies data
rain_anomalies = pd.read_table("Required data/Rainfall anomalies.txt", names=["FinYear", "Anomaly"])
rain_anomalies["FinYear"] = rain_anomalies["FinYear"].str[:4].astype(int)
rain_anomalies_sub = rain_anomalies[rain_anomalies["FinYear"].isin(Data6_kha_hist["FinYear"].unique())]

# Load RelCapacity
rel_capacity = pd.read_pickle("Results/Data/RelCapacity.pkl")

# Create RelCapacityBOM.aus dataset
rel_capacity_bom_aus = (
    rel_capacity
    .assign(FinYear=lambda x: pd.to_datetime(x["Date"]).dt.strftime("%Y-%m-%d"))
    .groupby("FinYear")["RelSurfaceArea"]
    .mean()
    .reset_index()
    .merge(rain_anomalies_sub, on="FinYear", how="inner")
)

rel_capacity_bom_aus.to_pickle("Results/Data/RelCapacityBOM_aus.pkl")

# Plot Predict.Plot.area.tot2.T3
plot_area_tot2_T3 = (
    ggplot(Data6_kha_hist.query("State != 'Australia'"), aes(x="FinYear", y="SumArea.kha.Adj.ecd"))
    + geom_area(aes(fill="State"))
    + labs(fill="", x="Financial Year")
    + theme_bw()
    + theme(
        panel_grid_major=None,
        panel_grid_minor=None,
        legend_position=(0.1, 0.85)
    )
    + scale_x_continuous(breaks=date_breaks("5 years"), labels=date_format("%Y"))
    + ggsave("Results/Plot/Predict.Plot.area2.T3.pdf", width=12.3, height=7)
)

plot_area_tot2_T3.save("Results/Plot/Predict.Plot.area2.T3.pdf", width=12.3, height=7)

plot_area_tot2_T3.show()

# Plot Predict.Plot.area.tot2.T3
plot_area_tot2_T3 = (
    ggplot(Data6_kha_hist.query("State != 'Australia'"), aes(x="FinYear", y="SumArea.kha.Adj.ecd"))
    + geom_area(aes(fill="State"))
    + geom_line(
        data=rain_anomalies_sub.query("FinYear > 1987"),
        aes(x="FinYear", y="Anomaly"),
        color="red",
        size=2
    )
    + labs(fill="", x="Financial Year")
    + theme_bw()
    + scale_x_continuous(breaks=date_breaks("5 years"), labels=date_format("%Y"))
    + scale_y_continuous(
        name="Historical increase in dam water surface area (area; kha)",
        sec_axis=sec_axis(trans="identity", name="Rainfall anomalies (line)")
    )
    + theme(
        panel_grid_major=None,
        panel_grid_minor=None,
        legend_position=(0.15, 0.85)
    )
)

plot_area_tot2_T3.save("Results/Plot/Predict.Plot.area3.T3.pdf", width=11.5, height=6.5)
plot_area_tot2_T3.show()

# Calculate Pearson's correlation
correlation = rel_capacity_bom_aus.query("FinYear > 1988")["RelSurfaceArea"].corr(
    rel_capacity_bom_aus.query("FinYear > 1988")["Anomaly"]
)
print("Pearson's correlation:", correlation)

# Lineplot of average relative capacity and BOM rainfall anomalies
plot_area_tot2_T4_a = (
    ggplot(rel_capacity_bom_aus.query("FinYear > 1988"), aes(x="FinYear", y="RelSurfaceArea"))
    + geom_line(color="black", size=1)
    + geom_line(
        data=rain_anomalies_sub.query("FinYear > 1988"),
        aes(x="FinYear", y=(rain_anomalies_sub["Anomaly"] / 4000) + 0.45),
        linetype="dashed"
    )
    + labs(x="Financial Year")
    + theme_bw()
    + scale_x_continuous(breaks=date_breaks("5 years"), labels=date_format("%Y"))
    + scale_y_continuous(
        name="Mean predicted water surface capacity (solid; %)",
        sec_axis=sec_axis(
            trans=lambda x: (x - 0.45) * 4000,
            name="Rainfall anomalies (dashed)"
        )
    )
    + theme(
        panel_grid_major=None,
        panel_grid_minor=None,
        legend_position=(0.15, 0.85)
    )
)

plot_area_tot2_T4_a.save("Results/Plot/Predict.Plot.area.tot2.T4.pdf", width=11.5, height=6.5)
plot_area_tot2_T4_a.show()

# Scatterplot of average relative capacity and BOM rainfall anomalies
plot_area_tot2_T4_b = (
    ggplot(rel_capacity_bom_aus.query("FinYear > 1988"), aes(x="Anomaly", y="RelSurfaceArea"))
    + geom_point(size=3)
    + geom_smooth(method="lm", se=False, color="black", linetype="dashed")
    + labs(x="Rainfall anomalies", y="Mean predicted water surface capacity (%)")
    + theme_bw()
    + theme(panel_grid_major=None, panel_grid_minor=None)
)

plot_area_tot2_T4_b.save("Results/Plot/Predict.Plot.area.tot2.T4.b.pdf", width=11.5, height=6.5)
plot_area_tot2_T4_b.show()

# Combine plots
plot_area_tot2_ab_T4 = plot_grid(plot_area_tot2_T4_a, plot_area_tot2_T4_b, ncol=1, labels=["A", "B"])
plot_area_tot2_ab_T4.save("Results/Plot/Predict.Plot.area.tot2.ab.T4.pdf", width=6.72, height=10.45)
plot_area_tot2_ab_T4.save("Results/Plot/Predict.Plot.area.tot2.ab.T4.jpeg", width=6.72, height=10.45)

# Load RelCapacityBOM.aus
rel_capacity_bom_aus = pd.read_pickle("Results/Data/RelCapacityBOM_aus.pkl")

SECTION 8

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from plotnine import ggplot, aes, geom_line, geom_ribbon, facet_wrap, labs, theme_bw, theme, scale_x_continuous
from plotnine import save_as_pdf_pages, save_as_jpeg_pages
from plotnine import save_plot
from mizani.breaks import date_breaks
from plotnine import ggtitle, guide_legend, guides

# Note: Due to the structure of the code, the content inside the "if(FALSE)" block is ignored in the Python version.

# Assuming Data6.kha.hist.Aus, Data6.kha.hist, and Data5.kha are available as DataFrames

# TotEmissions.Aus
TotEmissions_Aus = (
    ggplot(Data6.kha.hist.Aus, aes(x = pd.to_numeric(FinYear) + 1, y = TotE_T3_kt_year_Adj_ecd))
    + geom_line(show_legend = False)
    + geom_ribbon(aes(ymin = TotE_T3_kt_year_Adj_LCI_ecd, ymax = TotE_T3_kt_year_Adj_UCI_ecd), alpha = 0.4)
    + theme_bw()
    + labs(y = "Total methane emissions (CH$_4$ kt year$^{-1}$)", x = "Year")
    + theme(panel_grid_major = element_blank(), panel_grid_minor = element_blank(), legend_position = "top")
    + scale_x_continuous(limits = (1988, 2020))
)

save_plot(TotEmissions_Aus, filename = "Results/Plot/TotEmissions_Aus.pdf", width = 7, height = 7)
save_plot(TotEmissions_Aus, filename = "Results/Plot/TotEmissions_Aus.jpeg", width = 7, height = 7)


# TotEmissions (Divided by States)
TotEmissions = (
    ggplot(Data6.kha.hist, aes(x = pd.to_numeric(FinYear) + 1, y = TotE_T3_kt_year_Adj_ecd))
    + geom_line(show_legend = False)
    + geom_ribbon(aes(ymin = TotE_T3_kt_year_Adj_LCI_ecd, ymax = TotE_T3_kt_year_Adj_UCI_ecd), alpha = 0.4)
    + facet_wrap("State", scales = "free")
    + theme_bw()
    + labs(y = "Total methane emissions (CH$_4$ kt year$^{-1}$)", x = "Year")
    + theme(panel_grid_major = element_blank(), panel_grid_minor = element_blank(), legend_position = "bottom")
    + scale_x_continuous(limits = (1988, 2020))
)

save_plot(TotEmissions, filename = "Results/Plot/TotEmissions_State.pdf", width = 5.9, height = 5)
save_plot(TotEmissions, filename = "Results/Plot/TotEmissions_State.jpeg", width = 5.9, height = 5)


# TotEmissions2 (On the same graph as different lines)
TotEmissions2 = (
    ggplot(subset(Data6.kha.hist, State != "Australia"), aes(x = pd.to_numeric(FinYear) + 1, y = TotE_T3_kt_year_Adj_ecd))
    + geom_line(aes(color = "State"), size = 1.5)
    + theme_bw()
    + theme(panel_grid_major = element_blank(), panel_grid_minor = element_blank(), legend_position = "bottom")
    + labs(y = "Total methane emissions (CH$_4$ kt year$^{-1}$)", x = "Year", color = "State or Territory")
    + scale_x_continuous(limits = (1988, 2020))
)

save_plot(TotEmissions2, filename = "Results/Plot/TotEmissions_State2.pdf", width = 10.9, height = 8)
save_plot(TotEmissions2, filename = "Results/Plot/TotEmissions_State2.jpeg", width = 10.9, height = 8)


# TotEmissions_both
TotEmissions_both = (
    TotEmissions_Aus
    + TotEmissions2
    + plot_layout(ncols = 1, align = "hv", labels = ["A", "B"], hjust = -5, vjust = 2)
)

save_plot(TotEmissions_both, filename = "TotEmissions_both.pdf", base_width = 6.2, base_height = 8.2)
save_plot(TotEmissions_both, filename = "TotEmissions_both.jpeg", base_width = 6.2, base_height = 8.2)


# Comparison of total methane emissions across methods
Tier1 = Data6.kha.hist_Aus[Data6.kha.hist_Aus["FinYear"] == 2020].copy()
Tier1["Mean"] = 0.183 * Tier1["SumArea_kha_Adj_ecd"]
Tier1["LCI"] = 0.118 * Tier1["SumArea_kha_Adj_LCI_ecd"]
Tier1["UCI"] = 0.228 * Tier1["SumArea_kha_Adj_UCI_ecd"]
Tier1["Method"] = "Tier 1"

Tier3 = Data6.kha.hist_Aus[Data6.kha.hist_Aus["FinYear"] == 2020].copy()
Tier3.rename(columns={"TotE_T3_kt_year_Adj_ecd": "Mean", "TotE_T3_kt_year_Adj_LCI_ecd": "LCI", "TotE_T3_kt_year_Adj_UCI_ecd": "UCI"}, inplace=True)
Tier3["Method"] = "Tier 3"

# Approach for Tier2 (based on nls regression above)(search "nls")
# For the LCI, I will multiply the predicted log-10 value by 0.8432157
# For the UCI, I will multiply the predicted log-10 value by 1.100405
Tier2 = Data5.kha[(Data5.kha["FinYear"] == 2020) & (Data5.kha["State"] == "Australia")].copy()
Tier2["Mean"] = Tier2["TotE_T2_kt_year_Adj"].mean()
Tier2["LCI"] = 10 ** (0.8432157 * np.log10(Tier2["TotE_T2_kt_year_Adj"]))
Tier2["UCI"] = 10 ** (1.100405 * np.log10(Tier2["TotE_T2_kt_year_Adj"]))
Tier2["Method"] = "Tier 2"

# Create the final dataset
AllTiers = pd.concat([Tier1, Tier2, Tier3], ignore_index=True)

AllTiers.to_csv("Results/Data/AllTiers.csv", index=False)


SECTION 9

In [ ]:
# 9) ALL STATISTICS
# ===========

if False:

    # 4.1 Historical increase in agricultural ponds
    # ---

    # Australia average increase
    Incr1988to2020.Aus = Incr1988to2020[['Counts_1988', 'Counts_2020', 'SA_1988', 'SA_2020']].sum()
    Incr1988to2020.Aus['Rel_Count_1988to2020'] = (Incr1988to2020.Aus['Counts_2020'] - Incr1988to2020.Aus['Counts_1988']) / Incr1988to2020.Aus['Counts_1988']
    Incr1988to2020.Aus['Rel_SA_1988to2020'] = (Incr1988to2020.Aus['SA_2020'] - Incr1988to2020.Aus['SA_1988']) / Incr1988to2020.Aus['SA_1988']

    # State-by-State relative increases
    Incr1988to2020.sort_values(by='Rel_Count_1988to2020', ascending=False)


    # 4.2 Water surface area of agricultural ponds
    # ---

    # Plot year-to-year variability
    SA_YearToYear = ggplot(data=Data6.kha.hist, aes(x='FinYear', y='SumArea.kha.Adj')) +
                    geom_line(aes(color='State'))

    # Relative magnitude of year-to-year variability
    SA_YearToYear_Rel = (Data6.kha.hist.Aus['SumArea.kha.Adj'].diff() / Data6.kha.hist.Aus['SumArea.kha.Adj'].shift(1)).mean()
    SA_YearToYear_Rel = SA_YearToYear_Rel.dropna()
    100 * SA_YearToYear_Rel.abs().mean()
    SA_YearToYear_Rel.abs().max()
    SA_YearToYear_Rel.abs().sort_values()

    # Correlation with BOM rainfall anomalies
    corr_test_result = Data6.kha.hist.Aus[['RelCapacityBOM.aus.FinYear', 'RelCapacityBOM.aus.Anomaly']].dropna().corr(method='pearson')
    corr_test_result.loc['RelSurfaceArea', 'Anomaly']

    # Consequences of climate conditions on year-to-year surface area
    StatsAusSA = Data6.kha.hist.Aus[['FinYear', 'SumArea.kha.Adj.ecd']].dropna().diff()
    StatsAusSA['AbsDifSA'] = StatsAusSA['SumArea.kha.Adj.ecd'].abs()
    StatsAusSA['AbsDifSA'].describe()

    # Compare surface area and relative capacity between 2016 and 2018/2019
    Data6.kha.hist.Aus[['FinYear', 'SumArea.kha.Adj.ecd']].query('FinYear in [2016, 2018, 2019]')


    # 4.3 Methane flux from agricultural ponds
    # ---


    # 4.4 Total methane emissions from agricultural ponds in Australia
    # ---

    # Range and max
    Data6.kha.hist.Aus['TotE_T3.kt.year.Adj.ecd'].dropna().describe()

    # Trend
    Trend_Aus = np.polyfit(Data6.kha.hist.Aus['FinYear'], Data6.kha.hist.Aus['TotE_T3.kt.year.Adj.ecd'], 1)
    Trend_Aus = pd.Series(Trend_Aus, index=['Slope', 'Intercept'])
    Trend_Aus

    # Year-to-year variability
    year_to_year_variability = 100 * (0.07309 / 0.6438)

    # Is this increase due to stronger fluxes? No
    Trend_Aus_onlyClimate = np.polyfit(Data6.kha.hist.Aus['FinYear'], Data6.kha.hist.Aus['TotE_T3.kt.year.Adj'], 1)
    Trend_Aus_onlyClimate = pd.Series(Trend_Aus_onlyClimate, index=['Slope', 'Intercept'])
    Trend_Aus_onlyClimate

    # Is this increase due to larger dams? No
    Trend_Aus_onlySA = np.polyfit(Data6.kha.hist.Aus['FinYear'], Data6.kha.hist.Aus['SumArea.kha.Adj'], 1)
    Trend_Aus_onlySA = pd.Series(Trend_Aus_onlySA, index=['Slope', 'Intercept'])
    Trend_Aus_onlySA

    # Is this increase due to more dams being created? Yes
    Trend_Aus_num = np.polyfit(Data6.kha.hist.Aus['FinYear'], Data6.kha.hist.Aus['SumCount.Adj.ecd'], 1)
    Trend_Aus_num = pd.Series(Trend_Aus_num, index=['Slope', 'Intercept'])
    Trend_Aus_num

    # What is the effect on surface water?
    Trend_Aus_sw = np.polyfit(Data6.kha.hist.Aus['FinYear'], Data6.kha.hist.Aus['SumArea.kha.Adj.ecd'], 1)
    Trend_Aus_sw = pd.Series(Trend_Aus_sw, index=['Slope', 'Intercept'])
    Trend_Aus_sw


    # State by state
    subset(Data6.kha.hist, State == "New South Wales" & FinYear == 2020)['TotE_T3.kt.year.Adj.ecd']
    subset(Data6.kha.hist, State == "Queensland" & FinYear == 2020)['TotE_T3.kt.year.Adj.ecd']
    subset(Data6.kha.hist, State == "Victoria" & FinYear == 2020)['TotE_T3.kt.year.Adj.ecd']
    subset(Data6.kha.hist, State == "Western Australia" & FinYear == 2020)['TotE_T3.kt.year.Adj.ecd']



SECTION 10

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error as mae

# ------
# Average total methane emission at 15C
# ------

# Load the average farm dam emissions at 15C in mg.m2.day
# Assume the data is loaded from TotMet_15C.lm.RData in R
FarmDams_CH4kgyearha_at15C = 365.25 * 10000 * (10 ** fixef(TotMet_15C.lm)) / 1e+6
FarmDams_CH4kgyearha_at15C_LCI = 365.25 * 10000 * (10 ** confint(TotMet_15C.lm)[3, 0]) / 1e+6
FarmDams_CH4kgyearha_at15C_UCI = 365.25 * 10000 * (10 ** confint(TotMet_15C.lm)[3, 1]) / 1e+6

# CV for the relative error
Emissions_LCI = (FarmDams_CH4kgyearha_at15C - FarmDams_CH4kgyearha_at15C_LCI) / FarmDams_CH4kgyearha_at15C
Emissions_UCI = (FarmDams_CH4kgyearha_at15C_UCI - FarmDams_CH4kgyearha_at15C) / FarmDams_CH4kgyearha_at15C
Emissions_CV = np.mean(np.concatenate((Emissions_LCI, Emissions_UCI)))

# -------
# Farm dam area
# -------
if False:
    # Assume AreaVal.csv is read into a DataFrame named AreaVal
    AreaVal_lm = sm.OLS(AreaVal['Obs'], sm.add_constant(AreaVal['Pred'])).fit()
    AreaVal_lm_summary = AreaVal_lm.summary()

    # Calculate correlation
    correlation = AreaVal['Obs'].corr(AreaVal['Pred'])

    # MAPE: Mean Absolute Percentage Error
    mape = np.mean(np.abs((AreaVal['Obs'] - AreaVal['Pred']) / AreaVal['Obs'])) * 100

    # MAE: Mean Absolute Error
    mae_val = mae(AreaVal['Obs'], AreaVal['Pred']) / np.mean(AreaVal['Obs'])

    # Plot and save
    plt.figure()
    sns.scatterplot(x='Pred', y='Obs', data=AreaVal)
    sns.regplot(x='Pred', y='Obs', data=AreaVal, scatter=False, line_kws={"color": "red"})
    plt.annotate("Mean Absolute Perc. Error: 32.4%", xy=(3000, 16000), size=4.5)
    plt.savefig("Results/Plot/Validation_weather_to_water.jpeg")

    plt.figure()
    sns.scatterplot(x='Obs', y='Pred', data=AreaVal[AreaVal['Obs'] > 0])
    plt.xscale('log')
    plt.yscale('log')
    plt.savefig("Results/Plot/Validation_weather_to_water_log.jpeg")

# What about the error of measuring farm dams using satellite data?
if False:
    # Assume Together.csv is read into a DataFrame named AreaVal2
    AreaVal2_lm = sm.OLS(AreaVal2[AreaVal2['Type'] == "Measuring surface water with satellite data"]['Obs'],
                         sm.add_constant(AreaVal2[AreaVal2['Type'] == "Measuring surface water with satellite data"]['Pred'])).fit()

    mae_val = mae(AreaVal2[~np.isnan(AreaVal2['Obs'])]['Obs'],
                  AreaVal2[~np.isnan(AreaVal2['Obs'])]['Pred']) / np.mean(AreaVal2[~np.isnan(AreaVal2['Obs'])]['Obs'])

    # Plot and save
    g = sns.FacetGrid(AreaVal2, col='Type', col_wrap=3, sharex=False, sharey=False)
    g.map(sns.scatterplot, 'Obs', 'Pred')
    g.map(sns.regplot, 'Obs', 'Pred', scatter=False, line_kws={"color": "red"})
    g.savefig("Results/Plot/Validation_weather_to_water.jpeg")

# Decide on a CV
FarmDamArea_CV = 0.324 + 0.112

# -------
# Temp sensitivity
# -------
# Assume Boltzmann.lm.RData is loaded into a variable named TempSensitivity///
#YOU MUST LOAD THE BOLTZMAN.LM DATA/CHECK THE R SCRIPT FOR REFERENCE
TempEmissions = {'mean': TempSensitivity.params[1],
                 'LCI': confint(TempSensitivity)[1][0],
                 'UCI': confint(TempSensitivity)[1][1]}
TempEmissions_CV_LCI = (TempEmissions['mean'] - TempEmissions['LCI']) / TempEmissions['mean']
TempEmissions_CV_UCI = (TempEmissions['UCI'] - TempEmissions['mean']) / TempEmissions['mean']
TempEmissions_CV = np.mean([TempEmissions_CV_LCI, TempEmissions_CV_UCI])

# All CVs
CV = {'TempSensitivity': TempEmissions_CV, 'Area': FarmDamArea_CV, 'MeanEmissions': Emissions_CV}
labels = ['Temp. Sensitivity*\n(df = 251)', 'Water Surface \n(df = 12,235)', 'Methane Flux\n(df = 286)']

# Source of df:
# MetEmissions: length(FarmDamData$CH4_mgm2day.EbuAdj.15C) 286 (FigS2)
# TempSensitivity: length(RosData_nozeros$fch4_mgCH4m2d) 251 (FigS1)
# RatioData.CV: length(RatioData.data$Ratio_D_to_E) 162 (FigS3)
# Pond Area: weather-to-water model

AllCV = pd.DataFrame({'labels': labels, 'CV': list(CV.values())})
AllCV['CV.std'] = AllCV['CV'] / AllCV['CV'].sum()
AllCV['labels'] = pd.Categorical(AllCV['labels'], categories=labels, ordered=True)
AllCV.to_csv("Results/Data/AllCV.csv", index=False)

# Plot sources of uncertainty
plt.figure()
sns.barplot(x='labels', y='CV', data=AllCV)
plt.ylabel("Coefficient of variation of the mean")
plt.ylim(0, 1)
plt.savefig("Results/Plot/CV.pdf")
plt.savefig("Results/Plot/CV.jpeg")

# ----
# Statistics:
# ----
# All CVs
print(AllCV)

# Repercussions for
# Assume Data6.kha.hist.Aus is loaded into a DataFrame
print(Data6.kha.hist.Aus['TotE_T3.kt.year.Adj.ecd'])
print(Data6.kha.hist.Aus['TotE_T3.kt.year.Adj.LCI.ecd'])
print(Data6.kha.hist.Aus['TotE_T3.kt.year.Adj.UCI.ecd'])

# Fold difference in predictions
print(Data6.kha.hist.Aus['TotE_T3.kt.year.Adj.UCI.ecd'] / Data6.kha.hist.Aus['TotE_T3.kt.year.Adj.ecd'])
print(Data6.kha.hist.Aus['TotE_T3.kt.year.Adj.ecd'] / Data6.kha.hist.Aus['TotE_T3.kt.year.Adj.LCI.ecd'])

print(np.nanmin(np.array([Data6.kha.hist.Aus['TotE_T3.kt.year.Adj.UCI.ecd'] / Data6.kha.hist.Aus['TotE_T3.kt.year.Adj.ecd'],
                           Data6.kha.hist.Aus['TotE_T3.kt.year.Adj.ecd'] / Data6.kha.hist.Aus['TotE_T3.kt.year.Adj.LCI.ecd']])))

# Load AllCV from CSV
AllCV = pd.read_csv("Results/Data/AllCV.csv")
